# River Barrel Explorer

Explore river betting lines (triple barrels, flop-river checks, delayed barrels) to spot which holdings fuel bluffs versus value.


## Line Definitions

- **Triple Barrel** – player bet (or raised) on flop, turn, and barrelled the river.
- **Turn Follow-up** – player skipped the flop c-bet but bet (or raised) turn and followed with a river bet.
- **Flop-River** – player c-bet the flop, skipped betting the turn, then fired the river.
- **River Only** – player did not bet flop or turn, only led on the river.
- **Missed Draws** – any busted draw is counted under Air (tracked separately for drilldowns but not shown as its own row).


In [1]:
from pathlib import Path
import os

def _locate_project_root() -> Path:
    current = Path().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "AGENTS.md").exists():
            return candidate
    raise FileNotFoundError("Repository root not found from notebook location.")

PROJECT_ROOT = _locate_project_root()
del _locate_project_root

DB_CANDIDATES = [
    Path(r"T:\Dev\ignition\drivehud\drivehud.db"),
    Path("/mnt/t/Dev/ignition/drivehud/drivehud.db"),
    PROJECT_ROOT / "drivehud" / "drivehud.db",
]

for candidate in DB_CANDIDATES:
    if candidate.exists():
        DB_PATH = candidate
        break
else:
    checked = os.linesep.join(str(p) for p in DB_CANDIDATES)
    message = "Database not found. Checked:" + os.linesep + checked
    raise FileNotFoundError(message)

CACHE_PATH = PROJECT_ROOT / "analysis" / "cache" / "river_events.json"

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}")


In [2]:
import sys
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
import numpy as np

from analysis.river_utils import (
    RIVER_BUCKETS,
    available_primary_categories as available_river_categories,
    load_river_events,
    river_response_events,
)
from analysis.cbet_utils import BASE_PRIMARY_CATEGORIES
from matplotlib.colors import LinearSegmentedColormap


ModuleNotFoundError: No module named 'pandas'

In [ ]:

# --- Configuration ---
RIVER_BUCKET_BOUNDS = [
    (0.00, 0.25),
    (0.25, 0.40),
    (0.40, 0.60),
    (0.60, 0.80),
    (0.80, 1.00),
    (1.00, 1.25),
    (1.25, float('inf')),
]

def _bucket_label(low: float, high: float) -> str:
    if high == float('inf'):
        return f">={low:.2f}"
    return f"[{low:.2f}, {high:.2f})"

RIVER_BUCKETS = [(low, high, _bucket_label(low, high)) for (low, high) in RIVER_BUCKET_BOUNDS]
del _bucket_label

PRIMARY_GROUPS = {cat: [cat] for cat in BASE_PRIMARY_CATEGORIES}
PRIMARY_GROUPS['Missed Draw'] = ['Missed Draw']
GROUPED_PRIMARY_ORDER = [
    ('Events', None),
    ('Air', ['Air']),
    ('Missed Draw', ['Missed Draw']),
    ('Weak Pair', ['Underpair', 'Bottom Pair', 'Middle Pair']),
    ('Top Pair', ['Top Pair']),
    ('Overpair', ['Overpair']),
    ('Two Pair', ['Two Pair']),
    ('Trips/Set', ['Trips/Set']),
    ('Monster', ['Straight', 'Flush', 'Full House', 'Quads']),
]

FORCE_RELOAD = True


In [ ]:
events = load_river_events(DB_PATH, cache_path=CACHE_PATH, force=FORCE_RELOAD)
print(f"Loaded {len(events)} river bet events.")
print("Line types:", ", ".join(sorted({e["line_type"] for e in events})))
available_categories = available_river_categories(events)
print("Observed primary categories:", ", ".join(available_categories))

events_df = pd.DataFrame(events)
responses_df = pd.DataFrame(river_response_events(events))

bucket_labels = [b[2] for b in RIVER_BUCKETS]
bucket_edges = [b[0] for b in RIVER_BUCKETS] + [RIVER_BUCKETS[-1][1]]
events_df["bucket"] = pd.cut(events_df["ratio"], bins=bucket_edges, labels=bucket_labels, right=False)


In [ ]:
HAND_ORDER = ['Air'] + [cat for cat in BASE_PRIMARY_CATEGORIES if cat != 'Air']
LINE_DISPLAY_ORDER = ['Triple Barrel', 'Flop-River', 'Turn Follow-up', 'River Only', 'Other']

SPECIAL_FIELD_CANDIDATES = {
    'All-In': ('is_all_in', 'is_all_in_bet', 'is_all_in_river'),
    '1 BB': ('is_one_bb', 'is_one_bb_bet', 'is_one_bb_river'),
}


def _select_flag_field(df, candidates):
    for field in candidates:
        if field in df.columns:
            return field
    return None


def _subset_for_flag(df, candidates):
    field = _select_flag_field(df, candidates)
    if field is None:
        return None
    subset = df[df[field]]
    return subset if not subset.empty else None


def _style_bucket_table(summary_df, caption, special_cols):
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='YlOrRd', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(caption)
    )
    if special_cols:
        first = special_cols[0]
        idx = summary_df.columns.get_loc(first)
        styled = styled.set_table_styles([
            {'selector': f'th.col_heading.level0.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
            {'selector': f'td.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
        ], overwrite=False)
    return styled



def summarize_line(df):
    total = len(df)
    summary = {'Events': float(total)}
    for cat in HAND_ORDER:
        summary[cat] = df['primary'].eq(cat).mean() * 100 if total else 0.0
    return summary

def summarize_line_grouped(df):
    total = len(df)
    grouped = {'Events': float(total)}
    for label, members in GROUPED_PRIMARY_ORDER:
        if members is None or label == 'Events':
            continue
        grouped[label] = df['primary'].isin(members).mean() * 100 if total else 0.0
    return grouped

def display_line_comparison(df, caption):
    data = {}
    for line in LINE_DISPLAY_ORDER:
        subset = df[df['line_type'] == line]
        if subset.empty:
            continue
        data[line] = summarize_line(subset)
    if not data:
        print('No data for requested lines.')
        return
    summary_df = pd.DataFrame(data).reindex(['Events'] + HAND_ORDER)
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='YlOrRd', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(caption)
    )
    display(styled)

def display_line_comparison_grouped(df, caption):
    data = {}
    for line in LINE_DISPLAY_ORDER:
        subset = df[df['line_type'] == line]
        if subset.empty:
            continue
        data[line] = summarize_line_grouped(subset)
    if not data:
        print('No data for requested lines.')
        return
    summary_df = pd.DataFrame(data).reindex(['Events'] + [label for label, members in GROUPED_PRIMARY_ORDER if members is not None])
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='YlOrRd', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(caption)
    )
    display(styled)

def summarize_bucket(df):
    total = len(df)
    result = {'Events': float(total)}
    for cat in HAND_ORDER:
        result[cat] = df['primary'].eq(cat).mean() * 100 if total else 0.0
    return result

def summarize_bucket_grouped(df):
    total = len(df)
    result = {'Events': float(total)}
    for label, members in GROUPED_PRIMARY_ORDER:
        if members is None or label == 'Events':
            continue
        result[label] = df['primary'].isin(members).mean() * 100 if total else 0.0
    return result


def display_bucket_table(df, caption):
    bucket_labels_local = [bucket[2] for bucket in RIVER_BUCKETS]
    rows = []
    grouped_rows = []
    index = []
    for bucket_label in bucket_labels_local:
        subset = df[df['bucket'] == bucket_label]
        if subset.empty:
            continue
        rows.append(summarize_bucket(subset))
        grouped_rows.append(summarize_bucket_grouped(subset))
        index.append(bucket_label)
    if not rows:
        print('No data for selected line.')
        return
    summary_df = pd.DataFrame(rows, index=index).T
    summary_df = summary_df.reindex(['Events'] + HAND_ORDER)
    special_cols = []
    for label, candidates in SPECIAL_FIELD_CANDIDATES.items():
        subset = _subset_for_flag(df, candidates)
        if subset is None:
            continue
        series = pd.Series(summarize_bucket(subset))
        summary_df[label] = series.reindex(summary_df.index)
        special_cols.append(label)
    ordered_columns = [col for col in bucket_labels_local if col in summary_df.columns]
    ordered_columns += [col for col in special_cols if col in summary_df.columns]
    summary_df = summary_df[ordered_columns]
    display(_style_bucket_table(summary_df, caption, special_cols))

    grouped_df = pd.DataFrame(grouped_rows, index=index).T
    grouped_df = grouped_df.reindex(['Events'] + [label for label, members in GROUPED_PRIMARY_ORDER if members is not None])
    grouped_special = []
    for label, candidates in SPECIAL_FIELD_CANDIDATES.items():
        subset = _subset_for_flag(df, candidates)
        if subset is None:
            continue
        series = pd.Series(summarize_bucket_grouped(subset))
        grouped_df[label] = series.reindex(grouped_df.index)
        grouped_special.append(label)
    ordered_columns_grouped = [col for col in bucket_labels_local if col in grouped_df.columns]
    ordered_columns_grouped += [col for col in grouped_special if col in grouped_df.columns]
    grouped_df = grouped_df[ordered_columns_grouped]
    display(_style_bucket_table(grouped_df, f"{caption} (Grouped)", grouped_special))

def summarize_responses(df, response_type):
    df = df[df['response'] == response_type].dropna(subset=['responder_primary'])
    if df.empty:
        return None, None
    data = {}
    grouped = {}
    for line in ['Triple Barrel', 'Flop-River', 'Turn Follow-up', 'River Only']:
        subset = df[df['line_type'] == line]
        if subset.empty:
            continue
        total = len(subset)
        entry = {'Events': float(total)}
        entry_grouped = {'Events': float(total)}
        for cat in HAND_ORDER:
            entry[cat] = subset['responder_primary'].eq(cat).mean() * 100 if total else 0.0
        for label, members in GROUPED_PRIMARY_ORDER:
            if members is None or label == 'Events':
                continue
            entry_grouped[label] = subset['responder_primary'].isin(members).mean() * 100 if total else 0.0
        data[line] = entry
        grouped[line] = entry_grouped
    if not data:
        return None, None
    summary_df = pd.DataFrame(data).reindex(['Events'] + HAND_ORDER)
    grouped_df = pd.DataFrame(grouped).reindex(['Events'] + [label for label, members in GROUPED_PRIMARY_ORDER if members is not None])
    return summary_df, grouped_df

def display_response_table(response_type):
    summary_df, grouped_df = summarize_responses(responses_df, response_type)
    if summary_df is None:
        print(f'No responder data for {response_type} events.')
        return
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='PuBu', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(f'{response_type} vs River Bets (Responder Holdings)')
    )
    display(styled)

    grouped_styled = (
        grouped_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[grouped_df.index.difference(['Events']), :])
        .background_gradient(cmap='PuBu', axis=None, subset=pd.IndexSlice[grouped_df.index.difference(['Events']), :])
        .set_caption(f'{response_type} vs River Bets (Responder Holdings) (Grouped)')
    )
    display(grouped_styled)



In [ ]:


def _filter_standard_river_events(events_subset):
    if events_subset is None or events_subset.empty:
        return events_subset.iloc[0:0].copy()
    mask = pd.Series(True, index=events_subset.index)
    if 'is_all_in' in events_subset.columns:
        mask &= ~events_subset['is_all_in']
    if 'is_one_bb' in events_subset.columns:
        mask &= ~events_subset['is_one_bb']
    return events_subset.loc[mask].copy()


def _filter_standard_river_responses(responses_subset):
    if responses_subset is None or responses_subset.empty:
        return responses_subset.iloc[0:0].copy()
    return responses_subset.copy()


def _safe_mean(series):
    if series is None or len(series) == 0:
        return 0.0
    cleaned = series.dropna()
    if cleaned.empty:
        return 0.0
    value = cleaned.mean()
    return float(value) if pd.notna(value) else 0.0


def _prepare_river_event_metrics(events_subset, responses_subset, *, exclude_special=True):
    if events_subset is None or events_subset.empty:
        return pd.DataFrame()
    events_work = events_subset.copy()
    responses_work = responses_subset.copy()
    if exclude_special:
        events_work = _filter_standard_river_events(events_work)
    if events_work.empty:
        return pd.DataFrame()
    if exclude_special:
        responses_work = _filter_standard_river_responses(responses_work)
    events_work = events_work.rename(columns={'player': 'bettor'})
    base_cols = [
        'hand_number',
        'bettor',
        'line_type',
        'ratio',
        'bucket',
        'bet_amount',
        'bet_amount_bb',
        'big_blind',
        'pot_before',
        'bettor_in_position',
    ]
    missing = [col for col in base_cols if col not in events_work.columns]
    if missing:
        raise KeyError(f"Missing columns in events dataframe: {', '.join(missing)}")
    events_work = events_work[base_cols].copy()
    if responses_work.empty:
        grouped = pd.DataFrame(
            columns=[
                'hand_number',
                'bettor',
                'continue_amount',
                'call_amount',
                'raise_amount',
                'call_flag',
                'raise_flag',
            ]
        )
    else:
        allowed = events_work[['hand_number', 'bettor']].drop_duplicates()
        responses_work = responses_work.merge(allowed, on=['hand_number', 'bettor'], how='inner')
        if responses_work.empty:
            grouped = pd.DataFrame(
                columns=[
                    'hand_number',
                    'bettor',
                    'continue_amount',
                    'call_amount',
                    'raise_amount',
                    'call_flag',
                    'raise_flag',
                ]
            )
        else:
            prepped = responses_work[['hand_number', 'bettor', 'response', 'response_amount']].copy()
            prepped['response_amount'] = prepped['response_amount'].fillna(0.0)
            prepped['continue_amount'] = np.where(
                prepped['response'].isin(['Call', 'Raise']),
                prepped['response_amount'],
                0.0,
            )
            prepped['call_amount'] = np.where(
                prepped['response'] == 'Call',
                prepped['response_amount'],
                0.0,
            )
            prepped['raise_amount'] = np.where(
                prepped['response'] == 'Raise',
                prepped['response_amount'],
                0.0,
            )
            prepped['call_flag'] = (prepped['response'] == 'Call').astype(int)
            prepped['raise_flag'] = (prepped['response'] == 'Raise').astype(int)
            grouped = (
                prepped.groupby(['hand_number', 'bettor'], as_index=False)
                .agg({
                    'continue_amount': 'sum',
                    'call_amount': 'sum',
                    'raise_amount': 'sum',
                    'call_flag': 'max',
                    'raise_flag': 'max',
                })
            )
    metrics = events_work.merge(grouped, on=['hand_number', 'bettor'], how='left')
    for column in ['continue_amount', 'call_amount', 'raise_amount']:
        metrics[column] = metrics[column].fillna(0.0)
    for column in ['call_flag', 'raise_flag']:
        metrics[column] = metrics[column].fillna(0).astype(int)
    metrics['continue_flag'] = metrics['continue_amount'] > 0
    metrics['call_only_flag'] = (metrics['call_flag'] > 0) & (metrics['raise_flag'] == 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        metrics['bet_ratio_pct'] = metrics['ratio'].astype(float) * 100.0
        metrics['raise_ratio_pct'] = np.where(
            (metrics['raise_flag'] > 0) & (metrics['pot_before'] > 0),
            (metrics['raise_amount'] / metrics['pot_before']) * 100.0,
            np.nan,
        )
        metrics['breakeven_fold_pct'] = np.where(
            (metrics['bet_amount'] + metrics['pot_before']) > 0,
            metrics['bet_amount'] / (metrics['bet_amount'] + metrics['pot_before']) * 100.0,
            np.nan,
        )
    return metrics


def _compute_river_bucket_metrics(metrics_df, buckets=RIVER_BUCKETS):
    results = []
    if metrics_df is None or metrics_df.empty:
        for _, _, label in buckets:
            results.append({'Bucket': label, 'Events': 0.0})
        return results
    ratios = metrics_df['ratio'].astype(float)
    for (low, high, label) in buckets:
        if high == float('inf'):
            bucket_mask = ratios >= low
        else:
            bucket_mask = (ratios >= low) & (ratios < high)
        bucket_events = metrics_df.loc[bucket_mask]
        total = len(bucket_events)
        if total == 0:
            results.append({'Bucket': label, 'Events': 0.0})
            continue
        continue_events = int(bucket_events['continue_flag'].sum())
        raise_events = int(bucket_events['raise_flag'].sum())
        call_only_events = int(bucket_events['call_only_flag'].sum())
        continue_pct = continue_events / total * 100.0
        raise_pct = raise_events / total * 100.0
        call_only_pct = call_only_events / total * 100.0
        fold_pct = 100.0 - continue_pct
        avg_ratio_pct = _safe_mean(bucket_events['bet_ratio_pct'])
        raise_mask = bucket_events['raise_flag'] > 0
        avg_raise_ratio_pct = _safe_mean(bucket_events.loc[raise_mask, 'raise_ratio_pct'])
        avg_breakeven = _safe_mean(bucket_events['breakeven_fold_pct'])
        if np.isnan(avg_ratio_pct):
            avg_ratio_pct = 0.0
        if np.isnan(avg_raise_ratio_pct):
            avg_raise_ratio_pct = 0.0
        expected_call_contrib_pct = (call_only_pct * avg_ratio_pct) / 100.0
        expected_raise_contrib_pct = (raise_pct * avg_raise_ratio_pct) / 100.0
        total_contrib_pct = expected_call_contrib_pct + expected_raise_contrib_pct
        row = {
            'Bucket': label,
            'Events': float(total),
            'Fold%': fold_pct,
            'Continue%': continue_pct,
            'Call-Only%': call_only_pct,
            'Raise%': raise_pct,
            'Avg Size (Pot%)': avg_ratio_pct,
            'Avg Raise Size (Pot%)': avg_raise_ratio_pct,
            'Expected Call Contribution (Pot%)': expected_call_contrib_pct,
            'Expected Raise Contribution (Pot%)': expected_raise_contrib_pct,
            'Raise Contribution Total (Pot%)': total_contrib_pct,
            'Breakeven Fold%': avg_breakeven,
        }
        row['Fold Surplus'] = row['Fold%'] - row['Breakeven Fold%']
        results.append(row)
    return results


VALUE_COLUMNS_RIVER = [
    'Bucket',
    'Events',
    'Continue%',
    'Call-Only%',
    'Raise%',
    'Avg Size (Pot%)',
    'Avg Raise Size (Pot%)',
    'Expected Call Contribution (Pot%)',
    'Expected Raise Contribution (Pot%)',
    'Raise Contribution Total (Pot%)',
]


PRESSURE_COLUMNS_RIVER = [
    'Bucket',
    'Events',
    'Fold%',
    'Avg Size (Pot%)',
    'Breakeven Fold%',
    'Fold Surplus',
]


_FOLD_SURPLUS_CMAP_RIVER = LinearSegmentedColormap.from_list(
    'fold_surplus_cmp_river', ['#b91c1c', '#ffffff', '#15803d']
)


def _style_river_value_table(table_df, title):
    if table_df.empty:
        return None
    df = table_df.set_index('Bucket').T
    percent_rows = [row for row in df.index if row != 'Events']
    styled = df.style
    if 'Events' in df.index:
        styled = styled.format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
    if percent_rows:
        styled = styled.format('{:.1f}%', subset=pd.IndexSlice[percent_rows, :])
        contribution_rows = [row for row in percent_rows if 'Contribution' in row]
        size_rows = [row for row in percent_rows if 'Size' in row]
        pct_rows_other = [row for row in percent_rows if row not in contribution_rows + size_rows]
        if pct_rows_other:
            styled = styled.background_gradient(
                cmap='YlOrRd',
                axis=None,
                subset=pd.IndexSlice[pct_rows_other, :],
            )
        if size_rows:
            styled = styled.background_gradient(
                cmap='BuGn',
                axis=None,
                subset=pd.IndexSlice[size_rows, :],
            )
        if contribution_rows:
            styled = styled.background_gradient(
                cmap='PuBu',
                axis=None,
                subset=pd.IndexSlice[contribution_rows, :],
            )
    return styled.set_caption(title)


def _style_river_pressure_table(table_df, title):
    if table_df.empty:
        return None
    df = table_df.set_index('Bucket').T
    desired_order = ['Events', 'Fold%', 'Avg Size (Pot%)', 'Breakeven Fold%', 'Fold Surplus']
    existing_order = [row for row in desired_order if row in df.index]
    df = df.loc[existing_order]
    styled = df.style
    if 'Events' in df.index:
        styled = styled.format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
    percent_rows = [row for row in df.index if row != 'Events']
    if percent_rows:
        styled = styled.format('{:.1f}%', subset=pd.IndexSlice[percent_rows, :])
    if 'Fold%' in df.index:
        styled = styled.background_gradient(
            cmap='YlOrRd',
            axis=None,
            subset=pd.IndexSlice[['Fold%'], :],
        )
    if 'Avg Size (Pot%)' in df.index:
        styled = styled.background_gradient(
            cmap='BuGn',
            axis=None,
            subset=pd.IndexSlice[['Avg Size (Pot%)'], :],
        )
    if 'Fold Surplus' in df.index:
        fold_values = df.loc['Fold Surplus'].dropna().astype(float)
        if not fold_values.empty:
            max_abs = max(abs(fold_values.min()), abs(fold_values.max()))
            if max_abs == 0:
                max_abs = 1.0
            styled = styled.background_gradient(
                cmap=_FOLD_SURPLUS_CMAP_RIVER,
                axis=None,
                subset=pd.IndexSlice[['Fold Surplus'], :],
                vmin=-max_abs,
                vmax=max_abs,
            )
    return styled.set_caption(title)


def _display_river_event_tables(metrics_df, title_prefix):
    if metrics_df is None or metrics_df.empty:
        print(f'No events available for {title_prefix}.')
        return
    records = _compute_river_bucket_metrics(metrics_df)
    if not records:
        print(f'No bucketed metrics available for {title_prefix}.')
        return
    metrics_table = pd.DataFrame(records)
    value_columns = [col for col in VALUE_COLUMNS_RIVER if col in metrics_table.columns]
    pressure_columns = [col for col in PRESSURE_COLUMNS_RIVER if col in metrics_table.columns]
    value_table = metrics_table[value_columns]
    pressure_table = metrics_table[pressure_columns]
    value_styled = _style_river_value_table(value_table, f"{title_prefix} - Value Profile")
    if value_styled is not None:
        display(value_styled)
    pressure_styled = _style_river_pressure_table(pressure_table, f"{title_prefix} - Fold Pressure")
    if pressure_styled is not None:
        display(pressure_styled)


def display_river_event_ev_tables(line_type, title_prefix, *, exclude_special=True):
    line_events = events_df[events_df['line_type'] == line_type]
    if line_events.empty:
        print(f'No {line_type.lower()} events available for analysis.')
        return
    line_responses = responses_df[responses_df['line_type'] == line_type]
    base_metrics = _prepare_river_event_metrics(line_events, line_responses, exclude_special=exclude_special)
    if base_metrics.empty:
        print(f'No {line_type.lower()} events remain after filtering special cases.')
        return
    subsets = [
        (base_metrics, f"{title_prefix} (Overall)"),
        (base_metrics[base_metrics['bettor_in_position']], f"{title_prefix} (In Position)"),
        (base_metrics[~base_metrics['bettor_in_position']], f"{title_prefix} (Out of Position)"),
    ]
    for subset_df, label in subsets:
        if subset_df is None or subset_df.empty:
            continue
        _display_river_event_tables(subset_df, label)


def display_all_river_event_tables():
    if events_df.empty or responses_df.empty:
        print('No river events or responses available for event-level contribution tables.')
        return
    line_specs = [
        ('Triple Barrel', 'River Triple Barrel Event Outcomes'),
        ('Flop-River', 'River Flop-River Event Outcomes'),
        ('Turn Follow-up', 'River Turn Follow-up Event Outcomes'),
        ('River Only', 'River Only Event Outcomes'),
    ]
    for line_type, title in line_specs:
        display_river_event_ev_tables(line_type, title)



### River Event-Level Outcomes

River bet sizes summarised as pot-percent contributions with special cases removed.


In [ ]:
if events_df.empty or responses_df.empty:
    print('No river events or responses available for event-level analysis.')
else:
    display_all_river_event_tables()


In [ ]:
display_line_comparison(events_df, "River Hand Types by Line")
display_line_comparison_grouped(events_df, "River Hand Types by Line (Grouped)")


In [ ]:
display_bucket_table(events_df, 'River Bet Sizing (All Lines)')
display_bucket_table(events_df[events_df["line_type"] == "Triple Barrel"], "River Bet Sizing (Triple Barrel)")
display_bucket_table(events_df[events_df["line_type"] == "Flop-River"], "River Bet Sizing (Flop-River)")
display_bucket_table(events_df[events_df["line_type"] == "Turn Follow-up"], "River Bet Sizing (Turn Follow-up)")
display_bucket_table(events_df[events_df["line_type"] == "River Only"], "River Bet Sizing (River Only)")


In [ ]:
display_response_table("Call")
display_response_table("Raise")
